In [8]:
%run runaway_functionsv3
%matplotlib qt
cl = Cluster('NGC_4103')
cl.plot_cluster(extra=15)

<WCSAxesSubplot: >

In [43]:
from astropy.coordinates import Galactocentric
psrs = ATNF()
psrs_skycoord = psrs['SkyCoord']
# psrs_skycoord.transform_to(Galactocentric)
# (psrs_skycoord.cartesian.x.value,psrs_skycoord.cartesian.y.value,psrs_skycoord.cartesian.z.value)

In [ ]:
import numpy as np
import plotly.graph_objects as go
import pandas as pd
from astropy.coordinates import SkyCoord, Galactocentric
from astropy.table import Table

# Load data
dias2021 = Table.read("dias2021.tsv", format="ascii.ecsv")
maskplx = dias2021['Plx'] > 0.3
maskage = dias2021['logage'] < 7.7
clusters = dias2021[maskage & maskplx]
clusters['SkyCoord'] = SkyCoord(
    ra=clusters['RA_ICRS'],
    dec=clusters['DE_ICRS'],
    distance=clusters['Dist']
)

clusters['SkyCoord_galactocentric'] = clusters['SkyCoord'].transform_to(Galactocentric)
cl_galactocentric = clusters['SkyCoord_galactocentric']

x = cl_galactocentric.x
y = cl_galactocentric.y
z = cl_galactocentric.z

trace = go.Scatter3d(x=x,
                     y=y,
                     z=z,
                     text=clusters['Cluster'],
                     mode='markers',
                     marker=dict(color=clusters['logage'],
                                 size=3
                                 )
                     )

# Galaxy as a disk surface
radius = 15000
height = 2000  # Height of the disk

# Generate the grid for the disk
r = np.linspace(0, radius, num=100)
theta = np.linspace(0, 2 * np.pi, num=100)
r, theta = np.meshgrid(r, theta)

x = r * np.cos(theta)
y = r * np.sin(theta)
z = np.zeros_like(r)  # Flat disk

galaxy_surface = go.Surface(z=z, x=x, y=y, surfacecolor=x, colorscale='Blues', opacity=0.2)

# Plot
fig = go.Figure(data=[trace, galaxy_surface])
fig.update_layout(
    width=1000, 
    height=800,
    scene=dict(
        camera_eye=dict(x=0.6, y=0.6, z=0.6),
        aspectmode='manual',
        aspectratio=dict(x=1, y=1, z=0.1)
    )
)
fig.show()


In [ ]:
import numpy as np
import plotly.graph_objects as go
from PIL import Image
from astropy.coordinates import SkyCoord, Galactocentric
from astropy.table import Table
%run runaway_functionsv3

# Load data
dias2021 = Table.read("dias2021.tsv", format="ascii.ecsv")
maskplx = dias2021['Plx'] > 0.3
maskage = dias2021['logage'] < 7.7
clusters = dias2021[maskage & maskplx]
clusters['SkyCoord'] = SkyCoord(
    ra=clusters['RA_ICRS'],
    dec=clusters['DE_ICRS'],
    distance=clusters['Dist']
)

clusters['SkyCoord_galactocentric'] = clusters['SkyCoord'].transform_to(Galactocentric)
cl_galactocentric = clusters['SkyCoord_galactocentric']

x = cl_galactocentric.x
y = cl_galactocentric.y
z = cl_galactocentric.z

trace_clusters = go.Scatter3d(x=x,
                     y=y,
                     z=z,
                     text=clusters['Cluster'],
                     mode='markers',
                     marker=dict(
                        #  color=clusters['Dist'],
                                 size=2.5),
                     showlegend=False
                     )

# Load data
dias2021 = ATNF()
mask_dist = ~dias2021['DIST'].mask
clusters = dias2021[mask_dist]
clusters['SkyCoord'] = SkyCoord(
    ra=clusters['RAJD'],
    dec=clusters['DECJD'],
    distance=clusters['DIST']*1000
)

clusters['SkyCoord_galactocentric'] = clusters['SkyCoord'].transform_to(Galactocentric)
cl_galactocentric = clusters['SkyCoord_galactocentric']

x = cl_galactocentric.x
y = cl_galactocentric.y
z = cl_galactocentric.z

trace_psr = go.Scatter3d(x=x,
                     y=y,
                     z=z,
                     text=clusters['JNAME'],
                     mode='markers',
                     marker=dict(
                                #  color='cyan',
                                 size=5),
                     showlegend=False
                     )

# Load and process the image
image_path = 'mw.jpg'  # Update this to your image path
image = Image.open(image_path).convert('L')  # Convert to grayscale
# image = image.rotate(0, expand=True)  # Use -90 to rotate clockwise
image_width, image_height = image.size
image_array = np.array(image) / 255.0  # Normalize grayscale values to [0, 1]

# Galaxy as a disk surface with color gradient from image
radius = 21200
# height = 2000  # Height of the disk

# Generate the grid for the disk
r = np.linspace(0, radius, num=1000)
theta = np.linspace(0, 2 * np.pi, num=1000)
r, theta = np.meshgrid(r, theta)

x_disk = r * np.cos(theta)
y_disk = r * np.sin(theta)
z_disk = np.zeros_like(r)  # Flat disk

# Map disk coordinates to image coordinates
image_x = (x_disk / radius * (image_width / 2) + (image_width / 2)).astype(int)
image_y = (y_disk / radius * (image_height / 2) + (image_height / 2)).astype(int)

# Ensure coordinates are within image bounds
image_x = np.clip(image_x, 0, image_width - 1)
image_y = np.clip(image_y, 0, image_height - 1)

# Retrieve grayscale values from the image
colors = image_array[image_x, image_y]  # This gives the color for each point

# Define the surface with a color gradient based on image grayscale values
galaxy_surface = go.Surface(z=z_disk, x=x_disk, y=y_disk, surfacecolor=colors, colorscale='Gray', opacity=0.8,showscale=False)

sun = go.Scatter3d(x=[-8122],
                   y=[0],
                   z=[0],
                   text='Sun',
                   mode='markers',
                   marker=dict(color='yellow', size=5),
                   showlegend=False
                #    showscale=False
                   )

# Plot
fig = go.Figure(data=[trace_clusters, galaxy_surface, sun, trace_psr])
fig.update_layout(
    width=1800, 
    height=900,
        margin=dict(l=0, r=0, t=0, b=0),

    scene=dict(
        
        camera_eye=dict(x=1, y=0.6, z=0.6),
        aspectmode='manual',
        aspectratio=dict(x=1, y=1, z=1500/22000),
        xaxis=dict(
            title='Galactic Y (pc)',  # Rename X axis
            titlefont=dict(size=28),  # Increase font size
        ),
        yaxis=dict(
            title='Galactic X (pc)',  # Rename Y axis
            titlefont=dict(size=28),  # Increase font size
        ),
        zaxis=dict(
            title='Galactic Z (pc)',  # Rename Z axis
            titlefont=dict(size=28),  # Increase font size
        ),
    ),
    # paper_bgcolor='black',
    # plot_bgcolor='black'
)
fig.show()


In [ ]:
Isochrone()

In [14]:
%matplotlib qt
%run runaway_functionsv3

iso1 = get_theoretical_isochrone(Av=0,logage=5,FeH=0)
iso1 = get_theoretical_isochrone(Av=0,logage=6,FeH=0)
iso2 = get_theoretical_isochrone(Av=0,logage=7,FeH=0)
iso3 = get_theoretical_isochrone(Av=0,logage=8,FeH=0)
iso4 = get_theoretical_isochrone(Av=0,logage=9,FeH=0)
iso_dict = {
    1:iso1,
    2:iso2,
    3:iso3,
    4:iso4,
    5:iso5,
}


getting isochrone from cmd3.7 (PARSEC2) with:
Av:0.00
logage:5.00
metallicity:0.00
isochrone downloaded in 6.8s
getting isochrone from cmd3.7 (PARSEC2) with:
Av:0.00
logage:6.00
metallicity:0.00
isochrone downloaded in 6.0s
getting isochrone from cmd3.7 (PARSEC2) with:
Av:0.00
logage:7.00
metallicity:0.00
isochrone downloaded in 6.4s
getting isochrone from cmd3.7 (PARSEC2) with:
Av:0.00
logage:8.00
metallicity:0.00
isochrone downloaded in 6.0s
getting isochrone from cmd3.7 (PARSEC2) with:
Av:0.00
logage:9.00
metallicity:0.00
isochrone downloaded in 6.3s


In [15]:
fig, ax = plt.subplots(figsize=(10,10))
for i in range(1,6):
    ax.plot( iso_dict[i]['BP-RP'],iso_dict[i]['Gmag'], linewidth=5)
ax.invert_yaxis()


In [34]:
r50 = cl.all['r50']*u.deg.to(u.rad)
r50*1611

2.811725466860839

In [30]:
cl.all
5975542579654256640

Cluster,RA_ICRS,DE_ICRS,r50,N,pmRA,e_pmRA,pmDE,e_pmDE,Plx,e_Plx,RV,e_RV,NRV,Dist,e_Dist,logage,e_logage,__Fe_H_,e__Fe_H_,Av,e_Av,FileName,SimbadName,_RA.icrs,_DE.icrs
,deg,deg,deg,,mas / yr,mas / yr,mas / yr,mas / yr,mas,mas,km / s,km / s,,pc,pc,log(yr),log(yr),,,mag,mag,,,deg,deg
str17,float64,float64,float32,int16,float32,float32,float32,float32,float32,float32,float64,float32,int16,int16,int16,float32,float32,float32,float32,float32,float32,str31,str31,float64,float64
Bochum_13,259.3241,-35.5349,0.100,59,-0.282,0.154,-1.276,0.138,0.570,0.050,--,--,--,1611,61,6.900,0.035,0.299,0.063,2.800,0.040,clusters1/Bochum_13.dat,Cl Bochum 13,259.3241,-35.5349


In [92]:
cl = Cluster('RSG_8')
# cl.theoretical_isochrone

downloading sir
Searching in Gaia DR3 I/355/gaiadr3 34.811 arcmin around (<Longitude 11.824 deg>, <Latitude 85.2413 deg>, <Distance 1859. pc>)
found 2,664 sources in 0.29 seconds
Searching in Gaia DR3 distances I/352/gedr3dis 34.811 arcmin around (<Longitude 11.824 deg>, <Latitude 85.2413 deg>, <Distance 1859. pc>)
found 12,325 sources in 0.22 seconds
Starting merge of DR3 and distance catalog data
1,238 sources found by merging in 0.03 seconds
673 out of 881 dias members found in search region
Changed N          860.00 --> 637.00
Changed Plx        0.51 --> 0.53
Changed e_Plx      0.05 --> 0.03
Changed Dist       1859.00 --> 1823.00
Changed e_Dist     36.00 --> 104.24
Changed pmRA       -2.30 --> -2.32
Changed e_pmRA     0.18 --> 0.12
Changed pmDE       -0.95 --> -1.02
Changed e_pmDE     0.17 --> 0.12


AttributeError: 'Cluster' object has no attribute 'mymembers'

In [47]:
cl.theoretical_isochrone()

Mass,Teff0,BP-RP,Gmag,G_BP,G_RP,logg,logAge,logL,logTe,Mini,Gmag0
float64,float64,float64,float64,float64,float64,float64,float64,float64,float64,float64,float64
0.09,2525.0,4.321,21.85753113038979,24.61853113038979,20.297531130389793,4.117,7.15,-2.164,3.4023,0.0900000036,12.636
0.091,2529.0,4.321,21.84153113038979,24.602531130389792,20.281531130389794,4.118,7.15,-2.157,3.403,0.0910068527,12.62
0.098,2556.0,4.323,21.73253113038979,24.49453113038979,20.17153113038979,4.124,7.15,-2.114,3.4076,0.0976778716,12.511
0.1,2565.0,4.324,21.69453113038979,24.457531130389793,20.13353113038979,4.125,7.15,-2.098,3.4092,0.099999994,12.473
0.103,2578.0,4.311,21.632531130389793,24.384531130389792,20.073531130389792,4.128,7.15,-2.077,3.4114,0.1035206094,12.411
0.11,2603.0,4.271000000000001,21.50553113038979,24.224531130389792,19.95353113038979,4.132,7.15,-2.038,3.4155,0.1101747304,12.284
0.117,2628.0,4.23,21.376531130389793,24.061531130389792,19.83153113038979,4.136,7.15,-1.999,3.4196,0.1172624975,12.155
0.12,2637.0,4.205,21.31953113038979,23.983531130389792,19.778531130389794,4.137,7.15,-1.983,3.4212,0.1200000048,12.098
0.127,2660.0,4.131,21.17953113038979,23.781531130389794,19.650531130389794,4.141,7.15,-1.949,3.4249,0.1267942786,11.958


In [1]:
%matplotlib qt
%run runaway_functionsv3
dias2021

Cluster,RA_ICRS,DE_ICRS,r50,N,pmRA,e_pmRA,pmDE,e_pmDE,Plx,e_Plx,RV,e_RV,NRV,Dist,e_Dist,logage,e_logage,__Fe_H_,e__Fe_H_,Av,e_Av,FileName,SimbadName,_RA.icrs,_DE.icrs
,deg,deg,deg,,mas / yr,mas / yr,mas / yr,mas / yr,mas,mas,km / s,km / s,,pc,pc,log(yr),log(yr),,,mag,mag,,,deg,deg
str17,float64,float64,float32,int16,float32,float32,float32,float32,float32,float32,float64,float32,int16,int16,int16,float32,float32,float32,float32,float32,float32,str31,str31,float64,float64
ASCC_10,51.8737,35.0113,0.498,31,-1.704,0.168,-1.348,0.141,1.465,0.082,-10.870,5.243,3,643,30,8.178,0.155,0.109,0.100,0.820,0.110,clusters1/ASCC_10.dat,[KPR2005] 10,51.8737,35.0113
ASCC_101,288.4190,36.3653,0.357,72,0.924,0.272,1.364,0.289,2.502,0.068,-19.198,2.884,14,397,1,8.521,0.162,0.004,0.064,0.152,0.071,clusters1/ASCC_101.dat,[KPR2005] 101,288.4190,36.3653
ASCC_105,295.6291,27.3637,0.678,126,1.461,0.240,-1.611,0.199,1.786,0.066,-14.302,4.090,18,545,8,8.328,0.138,0.051,0.055,0.450,0.039,clusters1/ASCC_105.dat,[KPR2005] 105,295.6291,27.3637
ASCC_107,297.1623,22.0071,0.156,59,-0.144,0.135,-5.158,0.141,1.118,0.055,--,--,--,864,30,7.440,0.121,0.353,0.103,1.372,0.129,clusters1/ASCC_107.dat,[KPR2005] 107,297.1623,22.0071
ASCC_108,298.3162,39.3132,0.526,228,-0.509,0.101,-1.691,0.129,0.837,0.048,--,--,--,1138,31,8.334,0.108,0.093,0.093,0.464,0.071,clusters1/ASCC_108.dat,[KPR2005] 108,298.3162,39.3132
ASCC_11,53.0779,44.8787,0.285,274,0.950,0.196,-3.022,0.202,1.142,0.064,-14.242,0.233,3,849,20,8.660,0.063,-0.056,0.064,0.917,0.075,clusters1/ASCC_11.dat,[KPR2005] 11,53.0779,44.8787
ASCC_110,300.7723,33.5487,0.205,68,0.278,0.069,-3.134,0.075,0.495,0.036,-1.178,0.856,3,1583,143,8.795,0.084,0.140,0.195,1.228,0.122,clusters1/ASCC_110.dat,[KPR2005] 110,300.7723,33.5487
ASCC_111,302.7576,37.4883,0.520,153,-1.168,0.218,-1.525,0.163,1.168,0.063,-22.689,1.165,8,819,18,8.507,0.209,0.080,0.096,0.605,0.084,clusters1/ASCC_111.dat,[KPR2005] 111,302.7576,37.4883


In [20]:
d = 100*u.km/u.s*100*u.kyr
d = d.to(u.pc)
d

<Quantity 10.22712165 pc>

In [17]:
cl.Star(5333952049933787520)

Name,RA_ICRS_1,DE_ICRS_1,rgeo,b_rgeo,B_rgeo,Teff,Temp. Est,v_pec,e_v_pec,v_pec3d,HIP,TYC2,Source,Plx,e_Plx,pmRA,pmDE,e_pmRA,e_pmDE,RUWE,Gmag,BP-RP,BPmag,RPmag,e_Gmag,e_BPmag,e_RPmag,e_BP-RP,SkyCoord,rmRA,e_rmRA,rmDE,e_rmDE,logg,RV,e_RV,rRV,e_rRV,FG,e_FG,FBP,e_FBP,FRP,e_FRP,RAVE5,RAVE6
,deg,deg,pc,pc,pc,K,K,km / s,km / s,km / s,,,,mas,mas,mas / yr,mas / yr,mas / yr,mas / yr,,mag,mag,mag,mag,,,,,"deg,deg,pc",mas / yr,mas / yr,mas / yr,mas / yr,log(cm.s**-2),km / s,km / s,km / s,km / s,,,,,,,,
str9,float64,float64,float64,float64,float64,float64,float64,float64,float64,float64,int32,str11,int64,float64,float32,float64,float64,float32,float32,float64,float64,float64,float64,float64,float64,float32,float32,float32,SkyCoord,float64,float32,float64,float32,float64,float64,float32,float64,float32,float64,float32,float32,float32,float32,float32,str1,str1
HD 308823,174.84243990445,-63.36971443391,2539.82080000,1410.14795000,3852.90039000,15232.7,16210.0,48.530542153436116,32.98228162164111,--,--,--,5333952049933787520,1.9973,0.6318,-8.848,-2.165,0.668,0.721,32.069,11.291941,0.333353,11.389230,11.055877,0.002927,0.002866,0.003816,0.006682,"174.84243990445,-63.36971443391,2539.8208",-2.858,0.774,-2.843,0.827,3.5354,--,--,--,--,573020.91200,521.3,3.8e+05,228.9,2.998e+05,146.4,--,--
